In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import balanced_accuracy_score
import numpy as np
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# Завантаження даних
train_data = pd.read_csv("/kaggle/input/ml-fundamentals-and-applications-2024-10-01/final_proj_data.csv")
test_data = pd.read_csv("/kaggle/input/ml-fundamentals-and-applications-2024-10-01/final_proj_test.csv")

# Перевірка наявності цільової змінної
if 'y' not in train_data.columns:
    print("Помилка: Цільова змінна 'y' відсутня в наборі даних.")
else:
    # Визначення числових і категоріальних ознак
    numerical_cols = train_data.select_dtypes(include=['float64', 'int64']).columns.drop('y', errors='ignore')
    categorical_cols = train_data.select_dtypes(include=['object']).columns

    # Розподіл на ознаки та цільову змінну
    X = train_data.drop('y', axis=1)
    y = train_data['y']

    # Попередня обробка даних
    numerical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),  # Заповнення пропусків медіаною
        ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),  # Заповнення пропусків значенням 'missing'
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, numerical_cols),
            ('cat', categorical_transformer, categorical_cols)
        ])

    # Побудова ML-конвеєра з RandomForestClassifier
    model = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', RandomForestClassifier(class_weight='balanced', random_state=42))  # Балансування класів
    ])

    # Підбір гіперпараметрів
    param_grid = {
        'classifier__n_estimators': [100, 200, 300],
        'classifier__max_depth': [None, 10, 20, 30],
        'classifier__min_samples_split': [2, 5, 10],
        'classifier__min_samples_leaf': [1, 2, 4]
    }

    random_search = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=10, cv=5, scoring='balanced_accuracy', random_state=42, n_jobs=-1)
    
    # Розбиття на навчальну і тестову вибірки
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Підбір найкращої моделі
    random_search.fit(X_train, y_train)
    best_model = random_search.best_estimator_

    # Оцінка найкращої моделі
    y_pred = best_model.predict(X_val)
    print("Best Balanced Accuracy Score:", balanced_accuracy_score(y_val, y_pred))

    # Крос-валідація з найкращою моделлю
    cv_scores = cross_val_score(best_model, X, y, cv=5, scoring='balanced_accuracy')
    print("Середній результат крос-валідації:", np.mean(cv_scores))

    # Прогнози для тестового набору
    predictions = best_model.predict(test_data)

    # Формування файлу 
    output = pd.DataFrame({'index': test_data.index, 'y': predictions})
    output.to_csv("submission.csv", index=False)

    print("Файл з прогнозами збережено: 'submission.csv'")
    
    

/kaggle/input/ml-fundamentals-and-applications-2024-10-01/final_proj_data.csv
/kaggle/input/ml-fundamentals-and-applications-2024-10-01/final_proj_test.csv
/kaggle/input/ml-fundamentals-and-applications-2024-10-01/final_proj_sample_submission.csv
Best Balanced Accuracy Score: 0.7997432310854562
Середній результат крос-валідації: 0.7861954397537669
Файл з прогнозами збережено: 'submission.csv'


In [2]:
print(predictions.shape)

(2500,)
